4. Desarrollar una aplicación de software que calcule la Capacidad de Canal de un canal R-ario Uniforme y No Uniforme. El soft debe aceptar como entrada el valor de R que identifica al canal (R= 2 Binario, R=3 Ternario hasta R=4) los valores de probabilidades condicionales que representan la matriz del canal entregando como salida el valor de las probabilidades independientes de cada uno de los símbolos de entrada que maximiza la información mutua, esto es, lograr la capacidad de canal.

Se importan librerias

In [1]:
import numpy as np
from scipy.optimize import minimize

Calcula la información mutua I(X;Y) para una distribución de entrada Px y matriz de transición W (R x R).

In [2]:
def informacion_mutua(Px, W):
    Px = np.array(Px)
    W = np.array(W)
    Py = Px @ W  # distribución de salida
    I = 0.0
    for i in range(len(Px)):
        for j in range(len(Py)):
            if W[i, j] > 0 and Py[j] > 0:
                I += Px[i] * W[i, j] * np.log2(W[i, j] / Py[j])
    return I

Calcula la capacidad de canal para la matriz de transición W (R x R).     Devuelve Px óptimo y la capacidad.

In [3]:
def capacidad_canal(W):
    R = W.shape[0]

    # Función objetivo: negativo de I(X;Y) porque minimize() minimiza
    def objective(Px):
        return -informacion_mutua(Px, W)

    # Restricciones: sum(Px)=1, Px>=0
    constraints = ({
        'type': 'eq',
        'fun': lambda Px: np.sum(Px) - 1
    })
    bounds = [(0, 1)] * R

    # Punto inicial uniforme
    x0 = np.ones(R) / R

    result = minimize(objective, x0, bounds=bounds, constraints=constraints)

    if result.success:
        Px_opt = result.x
        C = informacion_mutua(Px_opt, W)
        return Px_opt, C
    else:
        raise ValueError("Optimización no convergió")

Programa principal.

In [4]:
if __name__ == "__main__":
    print("=== Cálculo de Capacidad de Canal ===")
    R = int(input("Ingrese el valor de R (2=Binario, 3=Ternario, 4=Cuaternario): "))
    while (R < 2 or R > 4):
        print("R debe ser 2, 3 o 4.")
        R = int(input("Ingrese el valor de R (2=Binario, 3=Ternario, 4=Cuaternario): "))

    print(f"Ingrese la matriz de probabilidades condicionales P(y|x), fila por fila.")
    print(f"Cada fila debe tener {R} valores separados por espacios y sumar 1.\n")
    P = []
    for i in range(R):
        fila = input(f"Fila {i} (para x={i}): ")
        P.append([float(x) for x in fila.strip().split()])
    P = np.array(P)

    Px_opt, C = capacidad_canal(P)
    print("Distribución óptima de entrada:", Px_opt)
    print("Capacidad del canal:", C, "bits")


=== Cálculo de Capacidad de Canal ===
Ingrese el valor de R (2=Binario, 3=Ternario, 4=Cuaternario): 2
Ingrese la matriz de probabilidades condicionales P(y|x), fila por fila.
Cada fila debe tener 2 valores separados por espacios y sumar 1.

Fila 0 (para x=0): 0.5 0.5
Fila 1 (para x=1): 0.4 0.6
Distribución óptima de entrada: [0.5 0.5]
Capacidad del canal: 0.007299156760473963 bits
